In [ ]:
pip install livelossplot


Mainly used this blog : https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/#:~:text=Fine%2DTuning%20Approach,-In%20this%20approach&text=The%20goal%20of%20fine%2Dtuning,outputting%20the%20transformed%20visual%20features.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten
from pathlib import Path
from livelossplot.inputs.keras import PlotLossesCallback
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:


BATCH_SIZE = 200

train_generator = ImageDataGenerator(
    rotation_range=90,
    brightness_range=[0.1, 0.7],
    width_shift_range=0.5,
    height_shift_range=0.5,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0,
    preprocessing_function=preprocess_input,
)  # VGG16 preprocessing

test_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input
)  # VGG16 preprocessing



In [ ]:
download_dir = Path("drive/MyDrive/task_1")
train_data_dir = download_dir / "train"
valid_data_dir = download_dir / "valid"
test_data_dir = download_dir / "test"

traingen = train_generator.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    class_mode="categorical",
    subset="training",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
)

train_generator = ImageDataGenerator(
    rotation_range=90,
    brightness_range=[0.1, 0.7],
    width_shift_range=0.5,
    height_shift_range=0.5,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.999,
    preprocessing_function=preprocess_input,
)  # VGG16 preprocessing


validgen = train_generator.flow_from_directory(
    valid_data_dir,
    target_size=(224, 224),
    class_mode="categorical",
    subset="validation",
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
)

testgen = test_generator.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    class_mode=None,
    batch_size=1,
    shuffle=False,
    seed=42,
)


In [ ]:
print(len(traingen))

In [ ]:

def create_model(input_shape, n_classes, optimizer="rmsprop", fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers

    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """

    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False, weights="imagenet", input_shape=input_shape)

    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation="relu")(top_model)
    top_model = Dense(1072, activation="relu")(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation="softmax")(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

    return model

In [ ]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes = 5

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 5

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)


plot_loss_1 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(
    filepath="drive/MyDrive/task_1/main_vgg.weights.best.keras", save_best_only=True, verbose=1
)

# EarlyStopping
early_stop = EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True, mode="min"
)



In [ ]:
print(vgg_model.summary())

In [ ]:
%%time


vgg_history = vgg_model.fit(
    traingen,
    batch_size=BATCH_SIZE,
    epochs=n_epochs,
    validation_data=validgen,
    steps_per_epoch=n_steps,
    validation_steps=n_val_steps,
    callbacks=[tl_checkpoint_1, early_stop, plot_loss_1],
    verbose=1,
)

In [ ]:
# Generate predictions
vgg_model.load_weights('drive/MyDrive/task_1/main_vgg.weights.best.keras') # initialize the best trained weights

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_preds = vgg_model.predict(testgen)
vgg_pred_classes = np.argmax(vgg_preds, axis=1)

In [ ]:
print(vgg_pred_classes)


In [ ]:
vgg_acc = accuracy_score(true_classes, vgg_pred_classes)
print("VGG16 Model Accuracy without Fine-Tuning: {:.2f}%".format(vgg_acc * 100))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
# Compute the confusion matrix
cm = confusion_matrix(true_classes, vgg_pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_indices.values(), yticklabels=class_indices.values())
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

Using finetuning


In [ ]:
# Reset our image data generators
traingen.reset()
validgen.reset()
testgen.reset()

# Use a smaller learning rate
optim_2 = Adam(learning_rate=0.01)
# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(
    filepath="drive/MyDrive/task_1/main_vgg_ft.weights.best.keras", save_best_only=True, verbose=1
)

# Re-compile the model, this time leaving the last 2 layers unfrozen for Fine-Tuning
vgg_model_ft = create_model(input_shape, n_classes, optim_2, fine_tune=2)

In [ ]:
print(len(validgen))

In [ ]:
%%time

plot_loss_2 = PlotLossesCallback()

# Retrain model with fine-tuning
vgg_ft_history = vgg_model_ft.fit(traingen,
                                  batch_size=BATCH_SIZE,
                                  epochs=n_epochs,
                                  validation_data=validgen,
                                  steps_per_epoch=n_steps,
                                  validation_steps=n_val_steps,
                                  callbacks=[tl_checkpoint_1, early_stop, plot_loss_2],
                                  verbose=1)

In [ ]:
# Generate predictions
vgg_model_ft.load_weights('drive/MyDrive/task_1/main_vgg_ft.weights.best.keras') # initialize the best trained weights

vgg_preds_ft = vgg_model_ft.predict(testgen)
vgg_pred_classes_ft = np.argmax(vgg_preds_ft, axis=1)

In [ ]:
true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

vgg_acc_ft = accuracy_score(true_classes, vgg_pred_classes_ft)
print("VGG16 Model Accuracy with Fine-Tuning: {:.2f}%".format(vgg_acc_ft * 100))

In [ ]:
print(vgg_model_ft.summary())